## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [13]:
DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0170.shtml"

Устанавливаем библиотеки

In [4]:
! pip install -q PyYaml==5.3.1 --user
! pip install -q rnnmorph==0.4.0
! pip install -q keras==2.1.4
! pip install -q h5py==2.7.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.1.4 which is incompatible.
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [1308 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9
  creating build\lib.win-amd64-3.9\h5py
  copying h5py\highlevel.py -> build\lib.win-amd64-3.9\h5py
  copying h5py\ipy_completer.py -> build\lib.win-amd64-3.9\h5py
  copying h5py\version.py -> build\lib.win-amd64-3.9\h5py
  copying h5py\__ini

In [5]:
! pip install rnnmorph

  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.1.4
    Uninstalling Keras-2.1.4:
      Successfully uninstalled Keras-2.1.4


Создаём объект морфологического анализатора `RNNMorph`

In [24]:
import warnings
warnings.filterwarnings('ignore')

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

In [23]:
!pip install rnnmorph

     ---------------------------------------- 19.7/19.7 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rnnmorph: filename=rnnmorph-0.4.1-py3-none-any.whl size=19746379 sha256=ae23d62ff4e6464f2c98f030fc47c3ec752a99c085f9dc061c1e38615a435b80
  Stored in directory: c:\users\сергей\appdata\local\pip\cache\wheels\72\a0\38\a38dd247132c2af64dec89a6575614d1c28b7a117374431123
Successfully built rnnmorph


Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [14]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [15]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Толстой Алексей Константинович. Упырь</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/t/tolstoj_a_k/>Толстой Алексей Константинович</a><br>\r\nУпырь</h2>'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [16]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [17]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Толстой Алексей Константинович. Упырь\n\n\n\nТолстой Алексей Константинович\r\nУпырь\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [18]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Сергей\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 1464 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [25]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться



sentences: 100%|██████████| 1464/1464 [00:00<00:00, 488579.01it/s]


[['слышать', 'ли', '?']]

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [26]:
len(predictions)

1464

In [27]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens) 


26488

'константинович'

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что']

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
